In [8]:
import os
import pandas as pd
import json

# Caminho para o arquivo JSON na pasta "data"
json_path = os.path.join("data", "sample_wanna_cry.json")  # Substitua "nome_do_arquivo.json" pelo nome real do arquivo

# Carrega o JSON a partir do caminho
with open(json_path, "r") as json_file:
    data_json = json.load(json_file)

# Extrai os resultados do JSON
results = data_json["results"]

# Lista para armazenar os indicadores
indicators_list = []

# Percorre os resultados e extrai todos os atributos dos indicadores
for result in results:
    for indicator in result["indicators"]:
        indicators_list.append({
            "indicator_id": indicator["id"],
            "indicator": indicator["indicator"],
            "indicator_type": indicator["type"],
            "indicator_created": indicator["created"],
            "indicator_content": indicator["content"],
            "indicator_title": indicator["title"],
            "indicator_description": indicator["description"],
            "indicator_expiration": indicator["expiration"],
            "indicator_is_active": indicator["is_active"]
        })

# Cria um DataFrame pandas com os indicadores
indicators_df = pd.DataFrame(indicators_list)

# Exibe o DataFrame
print(indicators_df)


      indicator_id                                          indicator   
0         12295635  09a46b3e1be080745a6d8d88d6b5bd351b1c7586ae0dc9...  \
1         12295636  24d004a104d4d54034dbcffc2a4b19a11f39008a575aa6...   
2         12295578  2584e1521065e45ec3c17767c065429038fc6291c09109...   
3         12295551  b9c5d4339809e0ad9a00d4d3dd26fdf44a32819a54abf8...   
4         12295579  ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439...   
...            ...                                                ...   
1551      21146762                   87be992695b752d86aeab1116eb5393f   
1552      21146763           0963034dfb7c0f703f5254595aea0a17ff95d376   
1553      21146764           7716a209006baa90227046e998b004468af2b1d6   
1554      21146765           b3a310c62632e25be611e08685cdf10c84ec722b   
1555      21146766                             4gxdnocmhl2tzx3z.onion   

       indicator_type    indicator_created indicator_content indicator_title   
0     FileHash-SHA256  2017-05-12T15:56:41 

In [11]:
import os
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Carrega o DataFrame a partir do script anterior
# Certifique-se de que o DataFrame "indicators_df" do script anterior ainda está disponível
# Se necessário, você pode recriar o DataFrame a partir do código anterior

# Seleciona as colunas relevantes para a análise
selected_columns = ["indicator_type", "indicator_created", "indicator_content", "indicator_title", "indicator_description"]

# Cria um novo DataFrame com as colunas selecionadas e faz uma cópia explícita
selected_indicators_df = indicators_df[selected_columns].copy()

# Converte a coluna "indicator_created" para o formato datetime
selected_indicators_df["indicator_created"] = pd.to_datetime(selected_indicators_df["indicator_created"])

# Preenche os valores ausentes com uma string vazia
selected_indicators_df.fillna("", inplace=True)

# Converte colunas de texto em variáveis categóricas
categorical_columns = ["indicator_type", "indicator_content", "indicator_title", "indicator_description"]
for column in categorical_columns:
    selected_indicators_df[column] = selected_indicators_df[column].astype("category")

# Transforma as variáveis categóricas em numéricas usando codificação one-hot
selected_indicators_encoded = pd.get_dummies(selected_indicators_df, columns=categorical_columns)

# Remove as colunas não importantes
selected_indicators_encoded.drop(columns=["indicator_is_active", "indicator_expiration", "indicator_id"], inplace=True)

# Divide os dados em conjunto de treinamento e teste
X = selected_indicators_encoded.drop(columns="indicator_type_FileHash-SHA256")
y = selected_indicators_encoded["indicator_type_FileHash-SHA256"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cria e treina o classificador Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Realiza a previsão
y_pred = gnb.predict(X_test)

# Calcula a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")

KeyError: "['indicator_is_active', 'indicator_expiration', 'indicator_id'] not found in axis"